In [1]:
from optimizer import Optimizer
from timeit import timeit

import warnings
warnings.filterwarnings('ignore')

# Performance Profiling

In [2]:
# These all need to available as global constants
covs=[1.793184, 0.704047, 5.865989]
iterations=1000

In [3]:
def profile_run():
    total_time = timeit("opt.get_demand_at_prices(covs)", number=iterations, globals=globals())
    print(f"get_demand_at_prices() avg runtime: {total_time / iterations:.6f} seconds")
    
    total_time = timeit("opt.get_revenue_maximizing_prices_and_revenue(demand)", number=iterations, globals=globals())
    print(f"get_revenue_maximizing_prices_and_revenue() avg runtime: {total_time / iterations:.6f} seconds")
    
    total_time = timeit("opt.get_revenue_maximizing_prices_and_revenue_from_cov(covs)", number=iterations, globals=globals())
    print(f"get_revenue_maximizing_prices_and_revenue_from_cov() avg runtime: {total_time / iterations:.6f} seconds")

## XGBoost

In [4]:
opt = Optimizer(model_file='xgbclassifier_v1.pkl', bought_range_p0=100, bought_range_p1=100)
demand = opt.get_demand_at_prices(covs)
profile_run()

get_demand_at_prices() avg runtime: 0.089825 seconds
get_revenue_maximizing_prices_and_revenue() avg runtime: 0.010742 seconds
get_revenue_maximizing_prices_and_revenue_from_cov() avg runtime: 0.097210 seconds


## Logistic Regression

In [5]:
opt = Optimizer(model_file='logit_v1.pkl', bought_range_p0=100, bought_range_p1=100)
demand = opt.get_demand_at_prices(covs)
profile_run()

get_demand_at_prices() avg runtime: 0.000781 seconds
get_revenue_maximizing_prices_and_revenue() avg runtime: 0.010003 seconds
get_revenue_maximizing_prices_and_revenue_from_cov() avg runtime: 0.010690 seconds


## GradientBoostingClassifier

In [6]:
opt = Optimizer(model_file='gradboost_v1.pkl', bought_range_p0=100, bought_range_p1=100)
demand = opt.get_demand_at_prices(covs)
profile_run()

get_demand_at_prices() avg runtime: 0.025221 seconds
get_revenue_maximizing_prices_and_revenue() avg runtime: 0.010278 seconds
get_revenue_maximizing_prices_and_revenue_from_cov() avg runtime: 0.035074 seconds
